In [29]:
import pandas as pd
import os
import glob
import numpy as np
from pathlib import Path

# Đường dẫn gốc đến thư mục chứa dữ liệu wilcoxon
BASE_DIR = r"D:\Optimal-Robust-Feature-Selection-For-Support-Vector-Machines-With-Pinball-Loss\src\experiment\results\wilcoxon"

# Danh sách các thành phần
ALL_DATASETS = ['wdbc', 'sonar', 'ionosphere', 'diabetes', 'cleveland']
DATASET_TYPES = ['outlier', 'original', 'noise', 'both'] 
ALL_MODELS = ['RFESVM', 'PinFSSVM', 'PinballSVM', 'MILP1', 'L2SVM', 'L1SVM', 'FisherSVM']

# CẤU HÌNH: Chọn dataset và models cần so sánh
TARGET_DATASET = 'diabetes'  # Thay đổi dataset ở đây
MODELS_TO_COMPARE = ['PinFSSVM', 'FisherSVM']  # Chọn 2 models để so sánh

print(f"Target Dataset: {TARGET_DATASET}")
print(f"Dataset Types: {len(DATASET_TYPES)} - {DATASET_TYPES}")
print(f"Models to compare: {MODELS_TO_COMPARE}")
print(f"Expected samples per model: {len(DATASET_TYPES)} * 10 folds = {len(DATASET_TYPES) * 10}")

def load_excel_file(file_path):
    """Load dữ liệu từ file Excel và trả về DataFrame"""
    try:
        if os.path.exists(file_path):
            df = pd.read_excel(file_path)
            return df
        else:
            print(f"  Warning: File not found: {file_path}")
            return None
    except Exception as e:
        print(f"  Error reading {file_path}: {e}")
        return None

def merge_single_dataset(dataset_name, models_list):
    """Merge dữ liệu cho 1 dataset cụ thể và danh sách models"""
    
    all_data = []
    missing_files = []
    
    print(f"\n--- Bắt đầu merge cho dataset: {dataset_name} ---")
    
    for model in models_list:
        print(f"\nProcessing model: {model}")
        model_data = []
        
        for dataset_type in DATASET_TYPES:
            # Tạo đường dẫn đến file
            file_path = os.path.join(BASE_DIR, dataset_name, dataset_type, f"{model}_auc_folds.xlsx")
            
            print(f"  Loading: {dataset_name}/{dataset_type}/{model}_auc_folds.xlsx")
            
            # Load dữ liệu
            df = load_excel_file(file_path)
            
            if df is not None:
                # Thêm thông tin metadata vào DataFrame
                df['Model'] = model
                df['Dataset'] = dataset_name
                df['Dataset_Type'] = dataset_type
                
                # Thêm vào danh sách
                model_data.append(df)
                all_data.append(df)
            else:
                missing_files.append(file_path)
        
        # Tính số samples cho model này
        total_samples_for_model = sum([len(df) for df in model_data])
        print(f"  Total samples for {model}: {total_samples_for_model}")
    
    # Merge tất cả dữ liệu
    if all_data:
        merged_df = pd.concat(all_data, ignore_index=True)
        print(f"\n--- Kết quả merge cho dataset {dataset_name} ---")
        print(f"Total DataFrame shape: {merged_df.shape}")
        print(f"Total missing files: {len(missing_files)}")
        
        return merged_df, missing_files
    else:
        print("No data to merge!")
        return None, missing_files

# Thực hiện merge cho dataset được chọn
merged_data, missing_files = merge_single_dataset(TARGET_DATASET, MODELS_TO_COMPARE)

if merged_data is not None:
    print("\n--- Thống kê dữ liệu merged ---")
    print(f"Columns: {list(merged_data.columns)}")
    print(f"Total rows: {len(merged_data)}")
    
    # Kiểm tra số lượng samples cho mỗi model
    print("\nSamples per model:")
    model_counts = merged_data['Model'].value_counts().sort_index()
    for model, count in model_counts.items():
        print(f"  {model}: {count} samples")
    
    # Kiểm tra số lượng samples cho mỗi dataset type
    print("\nSamples per dataset type:")
    type_counts = merged_data['Dataset_Type'].value_counts().sort_index()
    for dtype, count in type_counts.items():
        print(f"  {dtype}: {count} samples")
    
    # Hiển thị một vài dòng đầu
    print("\nFirst 10 rows:")
    print(merged_data.head(10))
    
    # Hiển thị thông tin về missing values
    print(f"\nMissing values per column:")
    print(merged_data.isnull().sum())

if missing_files:
    print(f"\n--- Missing files ({len(missing_files)}) ---")
    for file in missing_files:
        print(f"  {file}")

# Lưu dữ liệu merged vào file Excel
if 'merged_data' in locals() and merged_data is not None:
    
    # Tạo tên file output với tên dataset
    output_file = os.path.join(BASE_DIR, f"merged_{TARGET_DATASET}_data.xlsx")
    
    try:
        # Lưu vào Excel
        merged_data.to_excel(output_file, index=False)
        print(f"✅ Đã lưu dữ liệu merged vào: {output_file}")
        print(f"   Shape: {merged_data.shape}")
        
        # Tạo thêm một file CSV
        csv_output_file = os.path.join(BASE_DIR, f"merged_{TARGET_DATASET}_data.csv")
        merged_data.to_csv(csv_output_file, index=False)
        print(f"✅ Đã lưu dữ liệu merged vào: {csv_output_file}")
        
    except Exception as e:
        print(f"❌ Lỗi khi lưu file: {e}")

else:
    print("❌ Không có dữ liệu để lưu")

# So sánh 2 models cho dataset được chọn
if 'merged_data' in locals() and merged_data is not None and len(MODELS_TO_COMPARE) == 2:
    
    model1 = MODELS_TO_COMPARE[0]  # PinFSSVM
    model2 = MODELS_TO_COMPARE[1]  # RFESVM
    
    # Lấy dữ liệu cho 2 models
    model1_data = merged_data[merged_data['Model'] == model1].copy()
    model2_data = merged_data[merged_data['Model'] == model2].copy()
    
    print(f"--- So sánh {model1} vs {model2} trên dataset {TARGET_DATASET} ---")
    print(f"{model1} samples: {len(model1_data)}")
    print(f"{model2} samples: {len(model2_data)}")
    
    # Tạo cặp dữ liệu để so sánh (cùng dataset_type, fold)
    comparison_df = pd.merge(
        model1_data[['Dataset_Type', 'Fold', 'AUC']],
        model2_data[['Dataset_Type', 'Fold', 'AUC']],
        on=['Dataset_Type', 'Fold'],
        suffixes=(f'_{model1}', f'_{model2}')
    )
    
    print(f"\nPaired samples for comparison: {len(comparison_df)}")
    
    if len(comparison_df) > 0:
        # Hiển thị thống kê
        auc1_col = f'AUC_{model1}'
        auc2_col = f'AUC_{model2}'
        
        print(f"\nComparison Statistics:")
        print(f"{model1} AUC - Mean: {comparison_df[auc1_col].mean():.4f}, Std: {comparison_df[auc1_col].std():.4f}")
        print(f"{model2} AUC - Mean: {comparison_df[auc2_col].mean():.4f}, Std: {comparison_df[auc2_col].std():.4f}")
        
        # Tính differences
        comparison_df['Difference'] = comparison_df[auc1_col] - comparison_df[auc2_col]
        print(f"Difference ({model1} - {model2}) - Mean: {comparison_df['Difference'].mean():.4f}, Std: {comparison_df['Difference'].std():.4f}")
        
        # Hiển thị phân bố theo dataset_type
        print(f"\n--- Phân bố paired data theo Dataset_Type ---")
        distribution = comparison_df['Dataset_Type'].value_counts().sort_index()
        print(distribution)
        
        # Lưu dữ liệu so sánh
        comparison_output = os.path.join(BASE_DIR, f"comparison_{model1}_vs_{model2}_{TARGET_DATASET}.xlsx")
        comparison_df.to_excel(comparison_output, index=False)
        print(f"✅ Đã lưu dữ liệu so sánh vào: {comparison_output}")
        
        # Hiển thị một vài dòng đầu
        print(f"\nFirst 10 pairs:")
        print(comparison_df.head(10)[['Dataset_Type', 'Fold', auc1_col, auc2_col, 'Difference']])
        
        # Chuẩn bị cho Wilcoxon test
        model1_scores = comparison_df[auc1_col].values
        model2_scores = comparison_df[auc2_col].values
        
        print(f"\n--- Chuẩn bị cho Wilcoxon Test ---")
        print(f"Dataset: {TARGET_DATASET}")
        print(f"Total paired samples: {len(model1_scores)}")
        print(f"Expected samples: {len(DATASET_TYPES) * 10} = {len(DATASET_TYPES)} types × 10 folds")
        
        if len(model1_scores) == len(DATASET_TYPES) * 10:
            print("✅ Perfect! Có đúng số cặp dữ liệu mong đợi.")
        else:
            print(f"⚠️ Thiếu {len(DATASET_TYPES) * 10 - len(model1_scores)} cặp dữ liệu.")
    
    else:
        print("❌ Không tìm thấy cặp dữ liệu phù hợp để so sánh")

else:
    print("❌ Cần có đúng 2 models để so sánh hoặc không có dữ liệu merged")

# Thực hiện Wilcoxon test cho 1 dataset
from scipy.stats import wilcoxon

if 'model1_scores' in locals() and 'model2_scores' in locals():
    
    print(f"\n=== WILCOXON SIGNED-RANK TEST cho dataset {TARGET_DATASET} ===")
    print(f"So sánh: {MODELS_TO_COMPARE[0]} vs {MODELS_TO_COMPARE[1]}")
    print(f"Số cặp dữ liệu: {len(model1_scores)}")
    
    # Test 2-sided (kiểm tra có khác biệt hay không)
    statistic, p_value = wilcoxon(model1_scores, model2_scores, alternative='two-sided')
    
    print(f"\nKết quả test:")
    print(f"Test statistic: {statistic}")
    print(f"P-value (two-sided): {p_value:.6f}")
    
    alpha = 0.05
    if p_value < alpha:
        print(f"✅ Có sự khác biệt có ý nghĩa thống kê (p < {alpha})")
        
        # Kiểm tra model nào tốt hơn
        if np.mean(model1_scores) > np.mean(model2_scores):
            print(f"   {MODELS_TO_COMPARE[0]} có performance tốt hơn {MODELS_TO_COMPARE[1]}")
        else:
            print(f"   {MODELS_TO_COMPARE[1]} có performance tốt hơn {MODELS_TO_COMPARE[0]}")
    else:
        print(f"❌ Không có sự khác biệt có ý nghĩa thống kê (p >= {alpha})")
    
    # Test 1-sided để kiểm tra model1 có tốt hơn model2 không
    statistic_greater, p_value_greater = wilcoxon(model1_scores, model2_scores, alternative='greater')
    print(f"\nP-value ({MODELS_TO_COMPARE[0]} > {MODELS_TO_COMPARE[1]}): {p_value_greater:.6f}")
    
    if p_value_greater < alpha:
        print(f"✅ {MODELS_TO_COMPARE[0]} tốt hơn {MODELS_TO_COMPARE[1]} một cách có ý nghĩa thống kê")
    else:
        print(f"❌ Không thể khẳng định {MODELS_TO_COMPARE[0]} tốt hơn {MODELS_TO_COMPARE[1]}")

else:
    print("❌ Không có dữ liệu để thực hiện Wilcoxon test")

# Tóm tắt kết quả cho dataset
if 'merged_data' in locals() and merged_data is not None:
    
    print(f"=== TÓM TẮT KẾT QUẢ CHO DATASET {TARGET_DATASET} ===")
    
    # Thống kê tổng quan
    print(f"\nThống kê tổng quan:")
    print(f"Dataset: {TARGET_DATASET}")
    print(f"Models compared: {MODELS_TO_COMPARE}")
    print(f"Total records: {len(merged_data)}")
    print(f"Dataset types: {len(DATASET_TYPES)} - {DATASET_TYPES}")
    
    # Thống kê AUC cho từng model
    if 'AUC' in merged_data.columns:
        print(f"\nThống kê AUC cho từng model:")
        auc_stats = merged_data.groupby('Model')['AUC'].agg(['count', 'mean', 'std', 'min', 'max']).round(4)
        print(auc_stats)
        
        # Thống kê AUC theo Dataset_Type
        print(f"\nThống kê AUC theo Dataset_Type:")
        auc_by_type = merged_data.groupby(['Model', 'Dataset_Type'])['AUC'].agg(['mean', 'std']).round(4)
        print(auc_by_type)
    
    print(f"\n✅ Hoàn thành phân tích cho dataset {TARGET_DATASET}")

else:
    print("❌ Không có dữ liệu để tóm tắt")


Target Dataset: diabetes
Dataset Types: 4 - ['outlier', 'original', 'noise', 'both']
Models to compare: ['PinFSSVM', 'FisherSVM']
Expected samples per model: 4 * 10 folds = 40

--- Bắt đầu merge cho dataset: diabetes ---

Processing model: PinFSSVM
  Loading: diabetes/outlier/PinFSSVM_auc_folds.xlsx
  Loading: diabetes/original/PinFSSVM_auc_folds.xlsx
  Loading: diabetes/noise/PinFSSVM_auc_folds.xlsx
  Loading: diabetes/both/PinFSSVM_auc_folds.xlsx
  Total samples for PinFSSVM: 40

Processing model: FisherSVM
  Loading: diabetes/outlier/FisherSVM_auc_folds.xlsx
  Loading: diabetes/original/FisherSVM_auc_folds.xlsx
  Loading: diabetes/noise/FisherSVM_auc_folds.xlsx
  Loading: diabetes/both/FisherSVM_auc_folds.xlsx
  Total samples for FisherSVM: 40

--- Kết quả merge cho dataset diabetes ---
Total DataFrame shape: (80, 8)
Total missing files: 0

--- Thống kê dữ liệu merged ---
Columns: ['Model', 'Dataset', 'Dataset_Type', 'Fold', 'AUC', 'Best_C', 'Best_tau', 'Best_B']
Total rows: 80

Sam

In [30]:
# Import thư viện cần thiết
from scipy.stats import wilcoxon
import pandas as pd
import numpy as np

# OPTION 1: Wilcoxon test cho dataset hiện tại (TARGET_DATASET)
if 'merged_data' in locals() and merged_data is not None:
    
    print(f"=== WILCOXON SIGNED-RANK TEST FOR {TARGET_DATASET} ===")
    print(f"Comparing: {MODELS_TO_COMPARE[0]} vs {MODELS_TO_COMPARE[1]}")
    
    # Lấy dữ liệu cho 2 models
    model1_data = merged_data[merged_data['Model'] == MODELS_TO_COMPARE[0]]
    model2_data = merged_data[merged_data['Model'] == MODELS_TO_COMPARE[1]]
    
    print(f"\n--- Dataset: {TARGET_DATASET} ---")
    print(f"{MODELS_TO_COMPARE[0]} samples: {len(model1_data)}")
    print(f"{MODELS_TO_COMPARE[1]} samples: {len(model2_data)}")
    
    # Create paired comparison
    paired_comparison = pd.merge(
        model1_data[['Dataset_Type', 'Fold', 'AUC']],
        model2_data[['Dataset_Type', 'Fold', 'AUC']],
        on=['Dataset_Type', 'Fold'],
        suffixes=(f'_{MODELS_TO_COMPARE[0]}', f'_{MODELS_TO_COMPARE[1]}')
    )
    
    if len(paired_comparison) > 0:
        # Extract AUC scores
        scores1 = paired_comparison[f'AUC_{MODELS_TO_COMPARE[0]}'].values
        scores2 = paired_comparison[f'AUC_{MODELS_TO_COMPARE[1]}'].values
        
        print(f"Paired samples: {len(scores1)}")
        print(f"Mean {MODELS_TO_COMPARE[0]}: {scores1.mean():.4f}")
        print(f"Mean {MODELS_TO_COMPARE[1]}: {scores2.mean():.4f}")
        print(f"Mean difference: {(scores1 - scores2).mean():.4f}")
        
        # Perform Wilcoxon test
        try:
            # Two-sided test
            statistic, p_value_two = wilcoxon(scores1, scores2, alternative='two-sided')
            
            # One-sided test (model1 > model2)
            _, p_value_greater = wilcoxon(scores1, scores2, alternative='greater')
            
            # Determine significance
            alpha = 0.05
            significant = p_value_two < alpha
            model1_better = p_value_greater < alpha
            
            # Store results
            result = {
                'Dataset': TARGET_DATASET,
                'Model1': MODELS_TO_COMPARE[0],
                'Model2': MODELS_TO_COMPARE[1],
                'Pairs': len(scores1),
                'Mean1': scores1.mean(),
                'Mean2': scores2.mean(),
                'Mean_Diff': (scores1 - scores2).mean(),
                'Statistic': statistic,
                'P_value_two': p_value_two,
                'P_value_greater': p_value_greater,
                'Significant': significant,
                'Model1_Better': model1_better
            }
            
            # Print results
            print(f"Statistic: {statistic}")
            print(f"P-value (two-sided): {p_value_two:.6f}")
            print(f"P-value ({MODELS_TO_COMPARE[0]} > {MODELS_TO_COMPARE[1]}): {p_value_greater:.6f}")
            
            if significant:
                if model1_better:
                    print(f"✅ {MODELS_TO_COMPARE[0]} significantly better than {MODELS_TO_COMPARE[1]}")
                else:
                    print(f"✅ {MODELS_TO_COMPARE[1]} significantly better than {MODELS_TO_COMPARE[0]}")
            else:
                print(f"❌ No significant difference between models")
            
            # Create summary DataFrame
            wilcoxon_summary = pd.DataFrame([result])
            
            print("\n=== WILCOXON TEST SUMMARY ===")
            print(wilcoxon_summary[['Dataset', 'Mean1', 'Mean2', 'Mean_Diff', 'P_value_two', 'Significant', 'Model1_Better']].round(4))
            
            # Save results
            wilcoxon_output_file = os.path.join(BASE_DIR, f"wilcoxon_results_{MODELS_TO_COMPARE[0]}_vs_{MODELS_TO_COMPARE[1]}_{TARGET_DATASET}.xlsx")
            wilcoxon_summary.to_excel(wilcoxon_output_file, index=False)
            print(f"\n✅ Wilcoxon results saved to: {wilcoxon_output_file}")
                
        except Exception as e:
            print(f"Error in Wilcoxon test: {e}")
    
    else:
        print("❌ No paired data found for comparison")

else:
    print("❌ No merged data available for Wilcoxon test")


=== WILCOXON SIGNED-RANK TEST FOR diabetes ===
Comparing: PinFSSVM vs FisherSVM

--- Dataset: diabetes ---
PinFSSVM samples: 40
FisherSVM samples: 40
Paired samples: 40
Mean PinFSSVM: 0.7240
Mean FisherSVM: 0.7089
Mean difference: 0.0151
Statistic: 86.0
P-value (two-sided): 0.000299
P-value (PinFSSVM > FisherSVM): 0.000150
✅ PinFSSVM significantly better than FisherSVM

=== WILCOXON TEST SUMMARY ===
    Dataset  Mean1   Mean2  Mean_Diff  P_value_two  Significant  Model1_Better
0  diabetes  0.724  0.7089     0.0151       0.0003         True           True

✅ Wilcoxon results saved to: D:\Optimal-Robust-Feature-Selection-For-Support-Vector-Machines-With-Pinball-Loss\src\experiment\results\wilcoxon\wilcoxon_results_PinFSSVM_vs_FisherSVM_diabetes.xlsx
